In [4]:
from flask import Flask, request, jsonify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import os
from dotenv import load_dotenv

# Carica le credenziali di Spotify
load_dotenv()
SPOTIPY_CLIENT_ID = os.getenv("SPOTIPY_CLIENT_ID")
SPOTIPY_CLIENT_SECRET = os.getenv("SPOTIPY_CLIENT_SECRET")
auth_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [11]:
def get_track_features(track_id):
    # Ottieni le audio features per la traccia
    features = sp.audio_features(track_id)[0]
    if features:
        # Rimuovi le chiavi non necessarie
        features = {key: value for key, value in features.items() if key not in ['type', 'id', 'uri', 'track_href', 'analysis_url']}
    return features

In [32]:
def get_top_tracks(artist_id):
    results = sp.artist_top_tracks(artist_id)
    tracks = results['tracks'][:10]  # Limita a 10 le top tracks
    
    track_data = []
    for track in tracks:
        track_details = {
            'name': track['name'],
            'album': track['album']['name'],
            'artist': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id'],
            # 'artist_genres': track['artists'][0]['genres'],
            'release_date': track['album']['release_date'],
            'popularity': track['popularity'],
            'album_cover': track['album']['images'][0]['url'] if track['album']['images'] else None,
            'track_id': track['id']
        }
        # Aggiungi le audio features per ogni traccia
        track_features = get_track_features(track['id'])
        if track_features:
            track_details.update(track_features)
        
        track_data.append(track_details)
    
    return track_data



In [33]:
def artist_compare(artist_id_1,artist_id_2):
    # data = request.json
    # artist1_id = data.get(artist_id_1)
    # artist2_id = data.get(artist_id_2)
    
    artist1_tracks = get_top_tracks(artist_id_1)
    artist2_tracks = get_top_tracks(artist_id_2)
    
    df1 = pd.DataFrame(artist1_tracks)
    df2 = pd.DataFrame(artist2_tracks)
    
    combined_df = pd.concat([df1, df2])
    
    # Salva il DataFrame localmente
    combined_df.to_csv('top_tracks_with_selected_features.csv', index=False)
    
    return combined_df #jsonify({'message': 'Top tracks with selected audio features retrieved and saved successfully.', 'data': combined_df.to_dict(orient='records')})


In [34]:
artist_id_1 = "6RdcIWVKYYzNzjQRd3oyHS"
artist_id_2 = "06nvjg4wBANK6DCHjqtPNd"
results = artist_compare(artist_id_1, artist_id_2)

In [35]:
results

,name,album,artist,artist_id,release_date,popularity,album_cover,track_id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Nightmares,Nightmares,Bresh,7FeObngbQ0GY3SojNwKdKn,2023-09-29,71,https://i.scdn.co/image/ab67616d0000b2733b261d...,6SiJ1o6148rcmF2BydC1LW,0.714,0.833,...,-4.268,1,0.0378,0.4920,0.000000,0.2590,0.702,116.994,201782,3
1,Rubami la Notte,Fake News,Pinguini Tattici Nucleari,6RdcIWVKYYzNzjQRd3oyHS,2022-12-02,66,https://i.scdn.co/image/ab67616d0000b273df8e97...,3mMjLv3IyxlEKmXg3JxCI5,0.547,0.882,...,-5.187,1,0.0379,0.0671,0.000000,0.6130,0.642,77.011,189583,4
2,Ridere,Fuori dall'Hype Ringo Starr,Pinguini Tattici Nucleari,6RdcIWVKYYzNzjQRd3oyHS,2020-02-07,68,https://i.scdn.co/image/ab67616d0000b2739d4cb1...,5LZVK19ieaV8Mk9FsqTL3F,0.713,0.902,...,-3.009,1,0.0378,0.0493,0.000000,0.0781,0.720,116.909,214733,4
3,Giovani Wannabe,Fake News,Pinguini Tattici Nucleari,6RdcIWVKYYzNzjQRd3oyHS,2022-12-02,63,https://i.scdn.co/image/ab67616d0000b273df8e97...,4ICCU06b8FkQIrMv0XTzlv,0.742,0.809,...,-5.307,0,0.0309,0.0312,0.000000,0.0938,0.957,127.967,214280,4
4,Ricordi,Fake News,Pinguini Tattici Nucleari,6RdcIWVKYYzNzjQRd3oyHS,2022-12-02,64,https://i.scdn.co/image/ab67616d0000b273df8e97...,1AQqSrJYbCeRtzO7EApqYX,0.684,0.665,...,-5.987,0,0.0374,0.1230,0.000000,0.1530,0.609,135.018,203840,4
5,Scrivile Scemo,AHIA!,Pinguini Tattici Nucleari,6RdcIWVKYYzNzjQRd3oyHS,2020-12-04,66,https://i.scdn.co/image/ab67616d0000b27399db4e...,71r3UlkV7sWvUYjas5paqi,0.617,0.916,...,-3.538,1,0.0317,0.1230,0.000000,0.1830,0.552,135.091,214926,4
6,Ferma A Guardare (feat. Pinguini Tattici Nucle...,Gemelli,Ernia,3fhMfkPPzksWuw0hEm4ldm,2021-01-19,64,https://i.scdn.co/image/ab67616d0000b273816779...,1eDC4NiUYgQSKpKDIvXxi4,0.585,0.724,...,-5.611,0,0.0967,0.4450,0.000000,0.1000,0.547,162.050,213543,4
7,Ringo Starr,Fuori dall'Hype Ringo Starr,Pinguini Tattici Nucleari,6RdcIWVKYYzNzjQRd3oyHS,2020-02-07,63,https://i.scdn.co/image/ab67616d0000b2739d4cb1...,5yfE6GXTuJaAlepKoE0wJE,0.754,0.925,...,-2.912,0,0.0512,0.3040,0.000000,0.3130,0.857,114.077,183333,4
8,Scooby Doo,AHIA!,Pinguini Tattici Nucleari,6RdcIWVKYYzNzjQRd3oyHS,2020-12-04,63,https://i.scdn.co/image/ab67616d0000b27399db4e...,1P6TlFb8EVH2nCX3luVpZW,0.740,0.743,...,-4.166,1,0.1370,0.0477,0.000000,0.0925,0.773,135.071,179125,4
9,Pastello Bianco,AHIA!,Pinguini Tattici Nucleari,6RdcIWVKYYzNzjQRd3oyHS,2020-12-04,63,https://i.scdn.co/image/ab67616d0000b27399db4e...,0bKDQionn4VKcgNKbs4SWX,0.546,0.652,...,-3.852,1,0.0339,0.4510,0.000000,0.1230,0.378,139.875,236417,4
